# Discovering MlflowClient API 

- Create experiment
- List experiments
- Discovering experiment runs
- Manipulating runs info and data
- Model register
- Model versioning
- Testing models as a deployment engineer 
- Modify model alieses 

In [1]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

2025/06/14 04:47:18 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/14 04:47:18 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

### Experiments

In [ ]:
experiment_tags = {
  "developer": "kamal",
  "data": "nyc-taxi"
}

# created new experiment
# articat location parameter if None defaults to "mlfurn/{experiment_id}"
client.create_experiment("nyc-taxi-experiment", tags=experiment_tags)

'1'

In [7]:
# dicovering existing experiment
existing_experiments = client.search_experiments()

print(f"There exist {len(existing_experiments)} experiments")
for exp in sorted(existing_experiments, key=lambda exp: exp.experiment_id):
  print(f"{exp.experiment_id}: {exp.name}")
  for key, value in exp.tags.items():
    print(f"\t{key}: {value}")

There exist 2 experiments
0: Default
1: nyc-taxi-experiment
	developer: kamal
	data: nyc-taxi


### Creating some Runs

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
# This method should create the experiment if not exists
# But I created the experiment with same name on the prev step so mlflow will just use it
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

<Experiment: artifact_location='/Users/Kamal/WorkSpace/MLOps-zoomcamp/02. Experiment Tracking/mlruns/1', creation_time=1749865843928, experiment_id='1', last_update_time=1749865843928, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'data': 'nyc-taxi', 'developer': 'kamal'}>

In [10]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from data_loader import read_dataframe, encode_data

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/Kamal/WorkSpace/MLOps-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <3438F411-CE75-3B9D-AC3F-79994953DB2D> /Users/Kamal/WorkSpace/MLOps-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/

In [ ]:
features = ['PU_DO', 'trip_distance']
target = 'duration'

In [2]:
def list_experiments():
  for exp in client.search_experiments():
    print(f"id: {exp.experiment_id}, name: {exp.name}")

In [9]:
client.create_experiment(name="trial_x", tags={"developer": "kamal"})

'7'

In [3]:
list_experiments()

id: 7, name: trial_x
id: 3, name: mlflow_model_registry_practice_by_code
id: 2, name: mlflow_model_registry_practice


In [ ]:
runs = client.search_runs(
  filter_string="metrics.rmse < 8",
  run_view_type=ViewType.ACTIVE_ONLY,
  order_by=["metrics.rmse ASC"]
)

for run in runs:
  print(f"Id: {run.info.run_id}, name: {run.info.run_name}, rmse: {run.data.metrics['rmse']:.4f}")

Id: ea25f6ae8e1f4ad09071bf2f06c89b19, name: RandomForestRegressor, rmse: 6.9687
Id: 5ff37a319710474fb887b0f1688f83d2, name: DecisionTreeRegressor, rmse: 6.9795
Id: 7e4de4b900ae48a28eb1afb06bf8868c, name: LinearRegression, rmse: 7.7207


## Staging models

After discovering the deciding the best models, now you can register some models to the model_registery

In [5]:
REGISTERED_MODEL_NAME = "automatic_registry_1"

In [ ]:
client.create_registered_model(name=REGISTERED_MODEL_NAME)
client.al

<RegisteredModel: aliases={}, creation_timestamp=1749775356656, description=None, last_updated_timestamp=1749775356656, latest_versions=[], name='automatic_registry_1', tags={}>

In [ ]:
client.create_model_version(
  name=REGISTERED_MODEL_NAME + "XX", 
  source="runs:/7e4de4b900ae48a28eb1afb06bf8868c/model", 
  tags={"test": "hamada"}, 
  description="description test"
)


MlflowException: Registered Model with name=automatic_registry_1XX not found

In [8]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [25]:
for run in runs:
  # For every run with the same registered model a new version is created from the same ML-model
  mlflow.register_model(model_uri=f"runs:/{run.info.run_id}/model", name = REGISTERED_MODEL_NAME)

Registered model 'automatic_registry_1' already exists. Creating a new version of this model...
Created version '1' of model 'automatic_registry_1'.
Registered model 'automatic_registry_1' already exists. Creating a new version of this model...
Created version '2' of model 'automatic_registry_1'.
Registered model 'automatic_registry_1' already exists. Creating a new version of this model...
Created version '3' of model 'automatic_registry_1'.


In [32]:
client.set_registered_model_alias(REGISTERED_MODEL_NAME, "gamd_f45", 1)

In [36]:
client.get_model_version(REGISTERED_MODEL_NAME, 1).aliases

['gamd_f45', 'monster']

In [ ]:
client.update_model_version()